# Test Notebook for Building Pipeline Using Module 'concat_two_files'
The module 'concat_two_files' was generated through command: 'az ml module init -n concat_two_files'

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.wrapper import Module, Pipeline, dsl
import sys
from pathlib import Path
# The following line adds source directory to path. 
from concat_two_files_entry import concat_two_files

## Prerequisites: Configure workspace & compute

 - Update .azureml/config.json to make sure it has information on your workspace, subscription id, etc.
 - Change the following aml_compute_target's value to reference an existing compute target

In [ ]:
workspace = Workspace.from_config(path = '.azureml/config.json')
aml_compute_target = "YOUR_COMPUTE_TARGET"
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, aml_compute_target, sep='\n')

# Load module(s)
Load a module function through one of the following:
- Module.from_func: when you have a python function decorated with @dsl.module
- Module.from_yaml: when loading from existing module yaml definition
- Module.load: when you have already registered the module into Azure ML workspace

In [ ]:
module_func = Module.from_func(workspace, concat_two_files)
help(module_func)

# Run module in local
Using module.run can execute module in local machine<br>
If use_docker=True, will pull image from azure and run module in container.<br>
If use_docker=False, will directly run module script.

In [ ]:
# create module
module = module_func().set_parameters(
    append_line_break=False
).set_inputs(
    input1=str(Path('data') / 'concat_two_files_entry' / 'inputs' / 'input1'), input2=str(Path('data') / 'concat_two_files_entry' / 'inputs' / 'input2')
)
module.run(use_docker=True)

# Create pipeline
This is done by calling module function(s) with the parameters/inputs/outputs the module supports

In [ ]:
# prepare dataset with the following code 

# from azureml.core import Dataset

# your_dataset = Dataset.get_by_name(workspace, name='YOUR_DATASET_NAME')


In [ ]:
@dsl.pipeline(name='concat_two_files_pipeline', description='A sample pipeline uses concat_two_files', default_compute_target=aml_compute_target)
def sample_pipeline() -> Pipeline:
    module1 = module_func().set_parameters(
    
    ).set_inputs(
        # Please fill required inputs here
        # input1=None, input2=None
    )
    
    return module1.outputs

test_pipeline = sample_pipeline()

In [ ]:
test_pipeline.validate()

In [ ]:
run = test_pipeline.submit(
    experiment_name='concat_two_files_experiment'
)
run.wait_for_completion()
run